## Explaination
This script builds a condensed radar dataset at airport location

### About the weather radar data
Weather radar data is sourced from project rq0 on NCI. This data will soon be published via NCI (publically accessible by June 2020), but in the meantime you'll need to be an NCI user and request to join project rq0 to access the data. Data used for this exampled is processed 'level 2' data, which is on 1 x 1 km cartesian grid, rather than in the native radar coordinates (spherical). From the level 2 datasets, the 'reflectivity at 2km' and 'echo top heights' datasets are used. These consist of one netcdf file per radar per day per variable, whereby the dimensions are time, x and y.

### What are extracting?
In airport_table.csv, there is a radar identification number for each airport. For this radar, we are extracting the level 2 data within a search radius (in our case, 10 miles) from the airport. The maximum reflectivity is taken within this radius. A minimum threshold is also applied to remove precipitation that is likely not thunderstorms. If valid reflectivity of a thunderstorm exists within the search radius, the top of the reflectivity echo (proxy for storm top height) is also extracted.

### What does this data mean?
Reflectivity provides an indicator of thunderstorm presence and intensity
Echo Top Height provides additional information on the depth of a thunderstorm (which relates to the intensity)


In [1]:
#load libraries

import os
import glob
import warnings
from datetime import datetime, timedelta
from multiprocessing import Pool

import xarray as xr
import numpy as np
import pandas
import tqdm

In [2]:
#config
airport_csv_fn = 'airport_table.csv'
out_folder = '../preprocessed_data/'
radar_root = '/g/data/rq0/level_2/daily_150km'
start_date = '20100101'
end_date = '20181231'
search_radius = 16000 #m, using 10 mile radius
min_reflectivity = 50 #dBZ

In [3]:
def chunks(l, n):
    """
    Yield successive n-sized chunks from l.
    From http://stackoverflow.com/a/312464
    """
    for i in range(0, len(l), n):
        yield l[i:i + n]

def geographic_to_cartesian_aeqd(lon, lat, lon_0, lat_0, R=6370997.):
    """
    Azimuthal equidistant geographic to Cartesian coordinate transform.
    https://arm-doe.github.io/pyart/_modules/pyart/core/transforms.html#geographic_to_cartesian
    
    Transform a set of geographic coordinates (lat, lon) to
    Cartesian/Cartographic coordinates (x, y) using a azimuthal equidistant
    map projection [1]_.

    .. math::

        x = R * k * \\cos(lat) * \\sin(lon - lon_0)

        y = R * k * [\\cos(lat_0) * \\sin(lat) -
                     \\sin(lat_0) * \\cos(lat) * \\cos(lon - lon_0)]

        k = c / \\sin(c)

        c = \\arccos(\\sin(lat_0) * \\sin(lat) +
                     \\cos(lat_0) * \\cos(lat) * \\cos(lon - lon_0))

    Where x, y are the Cartesian position from the center of projection;
    lat, lon the corresponding latitude and longitude; lat_0, lon_0 are the
    latitude and longitude of the center of the projection; R is the radius of
    the earth (defaults to ~6371 km).

    Parameters
    ----------
    lon, lat : array-like
        Longitude and latitude coordinates in degrees.
    lon_0, lat_0 : float
        Longitude and latitude, in degrees, of the center of the projection.
    R : float, optional
        Earth radius in the same units as x and y. The default value is in
        units of meters.

    Returns
    -------
    x, y : array
        Cartesian coordinates in the same units as R, typically meters.

    References
    ----------
    .. [1] Snyder, J. P. Map Projections--A Working Manual. U. S. Geological
        Survey Professional Paper 1395, 1987, pp. 191-202.

    """
    lon = np.atleast_1d(np.asarray(lon))
    lat = np.atleast_1d(np.asarray(lat))

    lon_rad = np.deg2rad(lon)
    lat_rad = np.deg2rad(lat)

    lat_0_rad = np.deg2rad(lat_0)
    lon_0_rad = np.deg2rad(lon_0)

    lon_diff_rad = lon_rad - lon_0_rad

    # calculate the arccos after ensuring all values in valid domain, [-1, 1]
    arg_arccos = (np.sin(lat_0_rad) * np.sin(lat_rad) +
                  np.cos(lat_0_rad) * np.cos(lat_rad) * np.cos(lon_diff_rad))
    arg_arccos[arg_arccos > 1] = 1
    arg_arccos[arg_arccos < -1] = -1
    c = np.arccos(arg_arccos)
    with warnings.catch_warnings():
        # division by zero may occur here but is properly addressed below so
        # the warnings can be ignored
        warnings.simplefilter("ignore", RuntimeWarning)
        k = c / np.sin(c)
    # fix cases where k is undefined (c is zero), k should be 1
    k[c == 0] = 1

    x = R * k * np.cos(lat_rad) * np.sin(lon_diff_rad)
    y = R * k * (np.cos(lat_0_rad) * np.sin(lat_rad) -
                 np.sin(lat_0_rad) * np.cos(lat_rad) * np.cos(lon_diff_rad))
    return x, y

def read_csv(csv_ffn, header_line):
    """
    CSV reader used for the radar locations file (comma delimited)
    
    Parameters:
    ===========
        csv_ffn: str
            Full filename to csv file
            
        header_line: int or None
            to use first line of csv as header = 0, use None to use column index
            
    Returns:
    ========
        as_dict: dict
            csv columns are dictionary
    
    """
    df = pandas.read_csv(csv_ffn, header=header_line, skipinitialspace=True)
    as_dict = df.to_dict(orient='list')
    return as_dict

def daterange(date1, date2):
    """
    Generate date list between dates
    """
    date_list = []
    for n in range(int ((date2 - date1).days)+1):
        date_list.append(date1 + timedelta(n))
    return date_list

In [4]:
def extract_radar_data(radar_id, target_date, ap_lon, ap_lat, ap_name):
    
    #convert to strings
    radar_id_str = str(radar_id).zfill(2)
    target_date_str = datetime.strftime(target_date, '%Y%m%d')
    
    #print('radar data:',radar_id_str, target_date, ap_name)
    #check data files exist
    var = 'ECHO_TOP_HEIGHTS'
    eth_ffn = '/'.join([radar_root, var, radar_id_str, str(target_date.year)]) + '/' + '_'.join([radar_id_str, target_date_str, var]) + '.nc'
    var = 'REFLECTIVITY'
    ref_ffn = '/'.join([radar_root, var, radar_id_str, str(target_date.year)]) + '/' + '_'.join([radar_id_str, target_date_str, var]) + '.nc'
    if not os.path.isfile(eth_ffn) or not os.path.isfile(ref_ffn):
        return None
    #open datasets
    ds_eth = xr.open_dataset(eth_ffn)
    ds_ref = xr.open_dataset(ref_ffn)

    #find location of ap in radar cartesian coordinate space (x,y)
    radar_lat = float(ds_eth.origin_latitude)
    radar_lon = float(ds_eth.origin_longitude)
    ap_x, ap_y = geographic_to_cartesian_aeqd(ap_lon, ap_lat, radar_lon, radar_lat)
    #using x,y dimensions, calculate distance of every grid point from ap_x and ap_y
    x_grid, y_grid = np.meshgrid(ds_eth.x, ds_eth.y)
    dist_grid = np.sqrt((x_grid-ap_x)**2 + (y_grid-ap_y)**2)
    #find points within the search radius distance
    search_mask = dist_grid<search_radius
    #flag first pass as complete
    first_pass = False

    #maximum values
    search_mask_time = np.repeat(search_mask[np.newaxis, :, :], len(ds_ref.time), axis=0)
    ds_ref_search = ds_ref.reflectivity.where(search_mask_time, other=0) #replace everything outside search radius with 0
    ref_max = np.max(ds_ref_search, axis=(1,2))
    ds_eth_search = ds_eth.echo_top_heights.where(search_mask_time, other=0) #replace everything outside search radius with 0
    eth_max = np.max(ds_eth_search, axis=(1,2))
    #extract radar time
    radar_time_daily = ds_ref.time.data

    #threhold by reflectivity
    valid_mask = ref_max >= min_reflectivity
    #check if there's any valid data
    if np.any(valid_mask):
        #return arrays
        return {'time':radar_time_daily[valid_mask], 'ref':ref_max[valid_mask], 'eth':eth_max[valid_mask]}
    else:
        #no valid data, return nothing
        return None
    

In [5]:
#load airport list
ap_dict = read_csv(airport_csv_fn, header_line=1)
ap_name_list = ap_dict['Name']
ap_lat_list = ap_dict['Latitude']
ap_lon_list = ap_dict['Longitude']
ap_rid_list = ap_dict['radar_id']

In [ ]:
#radar data
date_list  = daterange(datetime.strptime(start_date, '%Y%m%d'), datetime.strptime(end_date, '%Y%m%d'))
NCPU = 15

#find radar reflectivity and echo tops for each airport
for i, ap_name in enumerate(ap_name_list):
    
    #initalise variables
    radar_time = np.array([],dtype='datetime64')
    radar_ref = np.array([])
    radar_eth = np.array([])
    
    #extract data from ap csv
    radar_id = ap_rid_list[i]
    ap_lat = ap_lat_list[i]
    ap_lon = ap_lon_list[i]
    
    #loop through dates using multiprocessing
    chunked_list  = chunks(date_list, NCPU)
    
    for list_slice in tqdm.tqdm(chunked_list, total=int(len(date_list)/NCPU)):
        with Pool(NCPU) as pool:
            args_list = [(radar_id, oneset, ap_lon, ap_lat, ap_name) for oneset in list_slice]
            result_list = pool.starmap(extract_radar_data, args_list)
            #compile results
            for result in result_list:
                if result is None:
                    continue
                else:
                    radar_time = np.append(radar_time, result['time'])
                    radar_ref = np.append(radar_ref, result['ref'])
                    radar_eth = np.append(radar_eth, result['eth'])

    #save to file
    print('finished', ap_name)
    save_path = out_folder + ap_name + '_radar.npz'
    np.savez(save_path, radar_ref=radar_ref, radar_eth=radar_eth, radar_time=radar_time)

220it [04:43,  1.29s/it]                         
  0%|          | 0/219 [00:00<?, ?it/s]

finished MEL


220it [04:47,  1.31s/it]                         
  0%|          | 0/219 [00:00<?, ?it/s]

finished SYD


 98%|█████████▊| 214/219 [04:41<00:06,  1.35s/it]